In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt
import mplleaflet
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json

In [10]:
gt_routes = pd.read_csv('google_transit/routes.txt', sep=",")
gt_routes

,route_id,route_short_name,route_long_name,route_desc,route_type,route_url
0,1-274,1,Queen,NaN,3,NaN
1,2-274,2,Main,NaN,3,NaN
2,3-274,3,McLaughlin,NaN,3,NaN
3,4-274,4,Chinguacousy,NaN,3,NaN
4,5-274,5,Bovaird,NaN,3,NaN
...,...,...,...,...,...,...
68,501-274,501,Zum Queen,NaN,3,NaN
69,502-274,502,Zum Main,NaN,3,NaN
70,505-274,505,Zum Bovaird,NaN,3,NaN
71,511-274,511,Zum Steeles,NaN,3,NaN
